In [1]:
from fastai import *
from fastai.callbacks import *
from fastai.vision import *


In [2]:
# load data from csv
# train_df = pd.read_csv('data/deep_fashion_data/csv/train_eval.csv', sep='\t')
# test_df = pd.read_csv('csv/test.csv', sep='\t')
# src = (
#     ImageList.from_df(
#         train_df,
#         path='data/deep_fashion_data',
#         cols='image_name',)
#     .split_from_df(col='is_valid')
#     .label_from_df(cols='category')
# )
# data = (
#     src.transform(get_transforms(flip_vert=True, max_warp=0.1, max_zoom=1.15, max_rotate=45.), size=150)
#     .databunch(num_workers=os.cpu_count(), bs=16,)
#     .normalize(imagenet_stats)
# )

In [26]:
# load data from csv
df = pd.read_csv("data/all_images_path.csv", sep='\t')
src = (
    ImageList.from_df(
        df,
        path='data',
        cols='path',)
    .split_none()
    # .split_by_rand_pct(0.001)
    .label_empty()
)
data = (
    src
    # .transform(get_transforms(flip_vert=True, max_warp=0.1, max_zoom=1.15, max_rotate=45.), size=150)
    .transform(size=150)
    .databunch(num_workers=os.cpu_count(), bs=256,)
    .normalize(imagenet_stats)
)
data.train_dl = data.train_dl.new(shuffle=False, drop_last=False)

In [27]:
data


ImageDataBunch;

Train: LabelList (204783 items)
x: ImageList
Image (3, 150, 150),Image (3, 150, 150),Image (3, 150, 150),Image (3, 150, 150),Image (3, 150, 150)
y: EmptyLabelList
,,,,
Path: data;

Valid: LabelList (0 items)
x: ImageList

y: EmptyLabelList

Path: data;

Test: None

In [28]:
# learner = cnn_learner(data, models.resnet50, path='.')
learner = load_learner(path='models/colab_export', file='res50_sz150_best_stage3_export.pkl')


/home/thanhlt998/anaconda3/envs/deep-learning/lib/python3.6/site-packages/torch/serialization.py:493: SourceChangeWarning: source code of class 'torch.nn.modules.loss.CrossEntropyLoss' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/thanhlt998/anaconda3/envs/deep-learning/lib/python3.6/site-packages/torch/serialization.py:493: SourceChangeWarning: source code of class 'torch.nn.modules.container.Sequential' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/thanhlt998/anaconda3/envs/deep-learning/lib/python3.6/site-packages/torch/serialization.py:493: SourceChangeWarning: source code of class 'torch.nn.modules.conv.Co

In [29]:
learner

Learner(data=ImageDataBunch;

Train: LabelList (0 items)
x: ImageList

y: CategoryList

Path: models/colab_export;

Valid: LabelList (0 items)
x: ImageList

y: CategoryList

Path: models/colab_export;

Test: None, model=Sequential(
  (0): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), s

In [30]:
feature_module = dict(learner.model.named_modules())['1.4']

In [31]:
feature_module

Linear(in_features=4096, out_features=512, bias=True)

In [32]:
class FeatureHook:
    def __init__(self, module: nn.Module, hook_func: HookFunc, forward: bool=True, detach: bool=True):
        self.hook_func, self.forward, self.detach = hook_func, forward, detach
        f = module.register_forward_hook if forward else module.register_backward_hook
        self.hook = f(self._hook)
        self.removed = False
        self.stored = None

    def _hook(self, module: nn.Module, input: Tensors, output: Tensors):
        if self.detach:
            input = (i.detach() for i in input) if is_listy(input) else input.detach()
            output = (o.detach() for o in output) if is_listy(output) else output.detach()
        hook_out = self.hook_func(module, input, output)
        self.stored = hook_out if self.stored is None else np.row_stack((self.stored, hook_out))

    def remove(self):
        if not self.removed:
            self.hook.remove()
            self.removed = True

    def __enter__(self):
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.remove()

In [33]:
def extract_features(module: nn.Module, input: Tensors, output: Tensors):
    return output.flatten(1).cpu().numpy()

In [34]:
with FeatureHook(feature_module, extract_features, forward=True, detach=True) as hook:
    # learner.model.eval()
    learner.validate(data.train_dl)
    output = hook.stored

In [35]:
np.save('features_resnet50.npy', output)

In [37]:
output.shape[0] == len(data.train_ds)

True